In [7]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.jsonl to train.jsonl
Saving val_wtihout (1).jsonl to val_wtihout (1).jsonl


In [ ]:
import pandas as pd
import json

In [ ]:
# функция, которая позволяет считать тренировочный набор данных и вернуть в виде Датафрейм
def get_X_y(data_json_file):
    X, y = [], []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        #print(json_list[0])
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            #print(item['passage'].keys())
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X.append(text+' Query: '+query+' Answer: '+a['text'])
                    y.append(a['label'])
    return X, y

X_train, y_train = get_X_y('train.jsonl')

train_df = pd.DataFrame({
    'text': X_train,
    'label': y_train
})

train_df

,text,label
0,"(1) Но люди не могут существовать без природы,...",1
1,"(1) Но люди не могут существовать без природы,...",0
2,"(1) Но люди не могут существовать без природы,...",0
3,"(1) Но люди не могут существовать без природы,...",1
4,"(1) Но люди не могут существовать без природы,...",0
...,...,...
11945,(1) Известный американский боец смешанных един...,0
11946,(1) Известный американский боец смешанных един...,1
11947,(1) Известный американский боец смешанных един...,0
11948,(1) Известный американский боец смешанных един...,0


In [8]:
# функция которая позволяет считать данные из теста и записать их в Датафрейм
def get_X_test(data_json_file):
    X1 = []
    X2 = []
    X3 = []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X1.append(text) #текст
                    X2.append(query) #вопрос
                    X3.append(a['text']) #ответ на вопрос
    return X1, X2, X3

X1, X2, X3 = get_X_test('val_wtihout (1).jsonl')

test_df = pd.DataFrame({
    'text': X1,
    'query': X2,
    'answer': X3
})

test_df

,text,query,answer
0,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,Так перевозили особо важных заключенных.
1,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что был эмигрантом."
2,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он сам вырыл себе землянку."
3,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он побывал на пересылке Красная Пр..."
4,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он был особо важным заключённым и ..."
...,...,...,...
2230,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Изнасилование.
2231,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Рокко похитил любовника и присвоил его деньги.
2232,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Убийство и расчленение тела.
2233,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Рокко убил любовника и расчленил его тело.


In [12]:
print(train_df.info())
print('--------------------------------------------------------------')
print(train_df.isna().sum())
print('-------------------------------------------------------')
print(train_df['label'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11950 entries, 0 to 11949
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    11950 non-null  object
 1   label   11950 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 186.8+ KB
None
--------------------------------------------------------------
text     0
label    0
dtype: int64
-------------------------------------------------------
0    6568
1    5382
Name: label, dtype: int64


In [13]:
from string import punctuation
import re

In [14]:
def remove_punct(text):
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ', 41: ' ', 42: ' ',
             43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ', 59: ' ', 60: ' ', 61: ' ', 62: ' ',
             63: ' ', 64: ' ', 91: ' ', 92: ' ', 93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

def txt_prep(df):

    df['text_01'] = df['text']
    df['text_01'] = df['text_01'].str.lower()
    # остаётся вопрос - нужно ли убирать 's или 'll - эти сокращения явно признак фейковости,
    # после чистки пунктуации там останется s - скорее всего проверка пунктуации определит это как ошибку
    df['text_01'] = df['text_01'].map(lambda x: remove_punct(x))

    signs = ['?', ',', ':', '!', '.', ';', '«', '»', '(', ')', '*', '+', '-', '^', '|', '=', "’", "‘",  '”', '—', '@', '>', '<', '–', '&', '%', '•', '/', '“']
    for punct_sign in signs:
        df['text_01'] = df['text_01'].str.replace(punct_sign, ' ')

    df['text_01'] = df['text_01'].str.replace('\n', '')
    df['text_01'] = df['text_01'].str.replace(r"\d+", "", flags=re.UNICODE)
    df['text_01'] = df['text_01'].str.replace("  " , " ")
    df['text_01'] = df['text_01'].str.replace("  " , " ")

    return df

In [15]:
train_df = txt_prep(train_df)

In [16]:
train_df

,text,label,text_01
0,"(1) Но люди не могут существовать без природы,...",1,но люди не могут существовать без природы поэ...
1,"(1) Но люди не могут существовать без природы,...",0,но люди не могут существовать без природы поэ...
2,"(1) Но люди не могут существовать без природы,...",0,но люди не могут существовать без природы поэ...
3,"(1) Но люди не могут существовать без природы,...",1,но люди не могут существовать без природы поэ...
4,"(1) Но люди не могут существовать без природы,...",0,но люди не могут существовать без природы поэ...
...,...,...,...
11945,(1) Известный американский боец смешанных един...,0,известный американский боец смешанных единобо...
11946,(1) Известный американский боец смешанных един...,1,известный американский боец смешанных единобо...
11947,(1) Известный американский боец смешанных един...,0,известный американский боец смешанных единобо...
11948,(1) Известный американский боец смешанных един...,0,известный американский боец смешанных единобо...


In [17]:
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
ma = MorphAnalyzer()

     |████████████████████████████████| 55 kB 1.4 MB/s 
     |████████████████████████████████| 8.2 MB 3.8 MB/s 


In [19]:
lemmatized_text = []
for i in range(len(train_df)):
    lemmatized_words = []

    text_in_words = train_df.loc[i]['text_01'].split(' ')

    for word in text_in_words:
        lemma = ma.parse(word)
        lemmatized_words.append(lemma[0].normal_form)

    text = " ".join(lemmatized_words)
    lemmatized_text.append(text)

train_df['text_02'] = lemmatized_text

In [20]:
train_df.head()

,text,label,text_01,text_02
0,"(1) Но люди не могут существовать без природы,...",1,но люди не могут существовать без природы поэ...,но человек не мочь существовать без природа п...
1,"(1) Но люди не могут существовать без природы,...",0,но люди не могут существовать без природы поэ...,но человек не мочь существовать без природа п...
2,"(1) Но люди не могут существовать без природы,...",0,но люди не могут существовать без природы поэ...,но человек не мочь существовать без природа п...
3,"(1) Но люди не могут существовать без природы,...",1,но люди не могут существовать без природы поэ...,но человек не мочь существовать без природа п...
4,"(1) Но люди не могут существовать без природы,...",0,но люди не могут существовать без природы поэ...,но человек не мочь существовать без природа п...


In [21]:
import nltk
from nltk.corpus import stopwords

In [22]:
nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

train_df['text_03'] = train_df['text_02']

for stop_word in russian_stopwords:
    stopword = r"\b" + stop_word + r"\b"
    train_df['text_03'] = train_df['text_03'].str.replace(stopword, '')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(train_df['text_03'], train_df['label'], test_size=0.3, random_state=42)

In [24]:
cv = CountVectorizer(ngram_range=(3,6), analyzer='word', max_features = 10000)

In [25]:
X_train_vector = cv.fit_transform(X_train).toarray()
X_test_vector = cv.transform(X_test).toarray()

In [26]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.1 MB/s 


In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import classification_report

In [51]:
models_names = ['DecisionTreeClassifier', 'SVC', 'KNeighborsClassifier', 'RandomForestClassifier', 'XGBClassifier', 'CatBoostClassifier']
models = [DecisionTreeClassifier(), SVC(random_state = 43), KNeighborsClassifier(), RandomForestClassifier(random_state = 43), XGBClassifier(), CatBoostClassifier(random_state = 43)]

In [52]:
for i in range(6):
    model = models[i]
    model_name = models_names[i]
    print('\n---------------------------------------------\n', model_name, '\n')
    model.fit(X_train_vector, y_train)
    predictions = model.predict(X_test_vector)
    print(classification_report(y_test, predictions))


---------------------------------------------
 DecisionTreeClassifier 

              precision    recall  f1-score   support

           0       0.52      0.90      0.66      1934
           1       0.28      0.05      0.08      1651

    accuracy                           0.50      3585
   macro avg       0.40      0.47      0.37      3585
weighted avg       0.41      0.50      0.39      3585


---------------------------------------------
 SVC 

              precision    recall  f1-score   support

           0       0.54      0.99      0.70      1934
           1       0.23      0.00      0.01      1651

    accuracy                           0.53      3585
   macro avg       0.38      0.50      0.35      3585
weighted avg       0.40      0.53      0.38      3585


---------------------------------------------
 KNeighborsClassifier 

              precision    recall  f1-score   support

           0       0.52      0.33      0.40      1934
           1       0.45      0.64      

In [ ]:
# Обработка теста

In [36]:
new_texts = []
#text	query	answer
for i in range(len(test_df)):
    text = test_df.loc[i]['text'] + test_df.loc[i]['query'] + test_df.loc[i]['answer']
    new_texts.append(text)

test = pd.DataFrame(data = new_texts, columns = ['text'])

In [55]:
test['text'][1]

'(1) Самый первый «остров» Архипелага возник в 1923 году на месте Соловецкого монастыря. (2) Затем появились ТОНы — тюрьмы особого назначения и этапы. (3) Люди попадали на Архипелаг разными способами: в вагон-заках, на баржах, пароходах и пешими этапами. (4) В тюрьмы арестованных доставляли в «воронках» — фургончиках чёрного цвета. (5) Роль портов Архипелага играли пересылки, временные лагеря, состоящие из палаток, землянок, бараков или участков земли под открытым небом. (6) На всех пересылках держать «политических» в узде помогали специально отобранные урки, или «социально близкие». (7) Солженицын побывал на пересылке Красная Пресня в 1945 году. (8) Эмигранты, крестьяне и «малые народы» перевозили красными эшелонами. (9) Чаще всего такие эшелоны останав\xadливались на пустом месте, посреди степи или тайги, и осуждённые сами строили лагерь. (10) Особо важные заключённые, в основном учёные, перевозились спецконвоем. (11) Так перевозили и Солженицына. (12) Он назвался ядерным физиком, и 

In [56]:
test = txt_prep(test)

In [57]:
lemmatized_text = []
for i in range(len(test)):
    lemmatized_words = []

    text_in_words = test.loc[i]['text_01'].split(' ')

    for word in text_in_words:
        lemma = ma.parse(word)
        lemmatized_words.append(lemma[0].normal_form)

    text = " ".join(lemmatized_words)
    lemmatized_text.append(text)

test['text_02'] = lemmatized_text

In [58]:
test['text_03'] = test['text_02']

for stop_word in russian_stopwords:
    stopword = r"\b" + stop_word + r"\b"
    test['text_03'] = test['text_03'].str.replace(stopword, '')

In [60]:
test_vector = cv.fit_transform(test['text_03']).toarray()

In [ ]:
#XGBClassifier 

In [62]:
res = models[4].predict(test_vector)

In [63]:
result = pd.DataFrame()

In [64]:
indexs = [i for i in range(len(res))]

In [67]:
result['index'] = indexs
result['label'] = res

In [70]:
result.to_csv('Dolzhenko_text_understanding.csv', index=False)
files.download("Dolzhenko_text_understanding.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>